In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')
import os
os.chdir("drive/My Drive/aimatch") 
!nvidia-smi
import torch
from torch import nn
print(f"\n cuda is available:　{torch.cuda.is_available()}",
      f"\n device count     :  {torch.cuda.device_count()}",
      f"\n device name      :  {torch.cuda.get_device_name(0)}",)

Thu Jun  4 05:09:33 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!python captcha_train_radam.py

init net
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value)
epoch: 0 step: 10 loss: 0.7630212593988808
epoch: 0 step: 20 loss: 0.7265285561157739
epoch: 0 step: 30 loss: 0.6802090687813603
epoch: 0 step: 40 loss: 0.625851287231116
epoch: 0 step: 50 loss: 0.5725389138704023
epoch: 0 step: 60 loss: 0.5305556525216885
epoch: 0 step: 70 loss: 0.4840668651571218
epoch: 0 step: 80 loss: 0.4501543777609649
epoch: 0 step: 90 loss: 0.41650301686030494
epoch: 0 step: 100 loss: 0.3815495145574555
epoch: 0 step: 110 loss: 0.3582948512673733
epoch: 0 step: 120 loss: 0.32124655800820084
epoch: 0 step: 130 loss: 0.2951253258117866
epoch: 0 step: 140 loss: 0.2754313491578646
epoch: 0 step: 150 loss: 0.24832769556968454
epoch: 0 step: 160 loss: 0.22642204098098703
epoch: 0 step: 

In [0]:
!python captcha_test.py

load cnn net.
Test Size: 2000
Test Accuracy of the model on the 200 test images: 95.000000 %
Test Accuracy of the model on the 400 test images: 95.500000 %
Test Accuracy of the model on the 600 test images: 95.333333 %
Test Accuracy of the model on the 800 test images: 95.625000 %
Test Accuracy of the model on the 1000 test images: 95.700000 %
Test Accuracy of the model on the 1200 test images: 96.166667 %
Test Accuracy of the model on the 1400 test images: 96.071429 %
Test Accuracy of the model on the 1600 test images: 95.750000 %
Test Accuracy of the model on the 1800 test images: 95.666667 %
Test Accuracy of the model on the 2000 test images: 95.600000 %
Test Accuracy of the model on the 2000 test images: 95.600000 %


In [0]:
!python captcha_predict.py

load cnn net.
Traceback (most recent call last):
  File "captcha_predict.py", line 71, in <module>
    main()
  File "captcha_predict.py", line 40, in main
    fopen = Image.open(file)
  File "/usr/local/lib/python3.6/dist-packages/PIL/Image.py", line 2818, in open
    prefix = fp.read(16)
KeyboardInterrupt


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader,Dataset,random_split
from torchvision import models
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

+ setting

In [0]:
#os.chdir("drive/My Drive/aimatch") 
NUMBER = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
UPPERCASE = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
LOWERCASE = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

ALL_CHAR_SET = NUMBER + UPPERCASE + LOWERCASE
ALL_CHAR_SET_LEN = len(ALL_CHAR_SET)
MAX_CAPTCHA = 4

# 图像大小
# IMAGE_HEIGHT = 60
# IMAGE_WIDTH = 160
IMAGE_HEIGHT = 40
IMAGE_WIDTH = 120

# 训练集、测试集、预测集的路径
TRAIN_DATASET_PATH = 'dataset' + os.path.sep + 'train'
TEST_DATASET_PATH = 'dataset' + os.path.sep + 'test'
PREDICT_DATASET_PATH = 'dataset' + os.path.sep + 'predict'

+ onehot编码
  

In [0]:
def encode(text):
    vector = np.zeros(ALL_CHAR_SET_LEN * MAX_CAPTCHA, dtype=float)
    def char2pos(c):
        if c =='_':
            k = 62
            return k
        k = ord(c)-48
        if k > 9:
            k = ord(c) - 65 + 10
            if k > 35:
                k = ord(c) - 97 + 26 + 10
                if k > 61:
                    raise ValueError('error')
        return k
    for i, c in enumerate(text):
        idx = i * ALL_CHAR_SET_LEN + char2pos(c)
        vector[idx] = 1.0
    return vector

def decode(vec):
    char_pos = vec.nonzero()[0]
    text=[]
    for i, c in enumerate(char_pos):
        char_at_pos = i #c/63
        char_idx = c % ALL_CHAR_SET_LEN
        if char_idx < 10:
            char_code = char_idx + ord('0')
        elif char_idx <36:
            char_code = char_idx - 10 + ord('A')
        elif char_idx < 62:
            char_code = char_idx - 36 + ord('a')
        elif char_idx == 62:
            char_code = ord('_')
        else:
            raise ValueError('error')
        text.append(chr(char_code))
    return "".join(text)

# e = encode("BK7H")
# print(e[62:124])
# print(decode(e))

+ 数据集加载

In [0]:
split_rate = 0.1 # 划分训练集和测试集的比率
torch.manual_seed(1) # 设置随机数种子为固定值

class mydataset(Dataset):
    def __init__(self, folder, transform=None):
        self.train_image_file_paths = [os.path.join(folder, image_file).replace('\\','/') for image_file in os.listdir(folder)]
        # print('train_image_file_paths:',self.train_image_file_paths)
        csv_path= 'dataset/train_label.csv'# os.path.join(folder,'train_label.csv').replace('\\','/') #dataset/train/train_label.csv
        data=pd.read_csv(csv_path)
        self.labels=np.array(data['label'])
        self.transform = transform

    def __len__(self):
        return len(self.train_image_file_paths)

    def __getitem__(self, idx):
        image_root = self.train_image_file_paths[idx]
        image = Image.open(image_root)
        image = image.resize((120, 40), Image.BICUBIC)      # 105*35 -> 120*40
        if self.transform is not None:
            image = self.transform(image)
        '''版本1：当图片文件名为标签名称时，通过'.'分割取出每张图片的标签存入label
        image_name = image_root.split('/')[-1]
        label = ohe.encode(image_name.split('.')[0]) # 为了方便，在生成图片的时候，图片文件的命名格式 "4个数字或者数字_时间戳.PNG", 4个字母或者即是图片的验证码的值，字母大写,同时对该值做 one-hot 处理
        '''
        # 版本2 保留原有的数据格式，通过csv文件获取每张图片标签而不用对每张图片进行重命名打上标签
        image_name = image_root.split('/')[-1] #xxxx.jpg
        image_num = image_name.split('.')[0] # 第xxxx张图片
        #print('图片名称',image_name,self.labels[int(image_num)-1])
        label=encode(self.labels[int(image_num)-1])
        return image, label

transform = transforms.Compose([
    # transforms.ColorJitter(),
    # transforms.Grayscale(),           # 可以尝试转灰度图处理，输入通道就要改成1
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 创建数据集对象
dataset = mydataset(TRAIN_DATASET_PATH, transform=transform)
# print(len(dataset))
# 测试集的长度
test_size = int(split_rate*len(dataset))
# print(test_size)
# 训练集的长度
train_size = int(len(dataset)-test_size)
# print(train_size)
# 随机划分
train_ds, test_ds = random_split(dataset, [train_size, test_size])

def get_train_data_loader(batch_size=64):
    # dataset = mydataset(captcha_setting.TRAIN_DATASET_PATH, transform=transform)
    # return DataLoader(dataset, batch_size=64, shuffle=True)
    return DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=16)

def get_test_data_loader():
    # dataset = mydataset(captcha_setting.TEST_DATASET_PATH, transform=transform)
    # return DataLoader(dataset, batch_size=1, shuffle=True)
    return DataLoader(test_ds, batch_size=1, shuffle=True, num_workers=4)

# def get_predict_data_loader():
#     dataset = mydataset(captcha_setting.PREDICT_DATASET_PATH, transform=transform)
#     return DataLoader(dataset, batch_size=1, shuffle=False)

'''这个方法不用了，Predict模块自写了顺序遍历'''
def get_predict_data_loader():
    dataset = mydataset(PREDICT_DATASET_PATH, transform=transform)
    return DataLoader(dataset, batch_size=1, shuffle=False)


test_loader = get_test_data_loader()
for i, (im, lb) in enumerate(test_loader):
    print(i, im[i].shape, lb[i].shape)
    if i == 0: break

0 torch.Size([3, 40, 120]) torch.Size([248])


+ 模型准确率

In [0]:
def test_data(model):
    model.eval()
    test_dataloader = get_test_data_loader()

    correct = 0
    total = 0

    for i, (images, labels) in enumerate(test_dataloader):
        image = images
        vimage = Variable(image).cuda()
        predict_label = model(vimage).cuda()

        c0 = ALL_CHAR_SET[np.argmax(
            predict_label[0, 0:ALL_CHAR_SET_LEN].data.cpu().numpy())]
        c1 = ALL_CHAR_SET[np.argmax(
            predict_label[0, ALL_CHAR_SET_LEN:2 * ALL_CHAR_SET_LEN].data.cpu().numpy())]
        c2 = ALL_CHAR_SET[np.argmax(
            predict_label[0, 2 * ALL_CHAR_SET_LEN:3 * ALL_CHAR_SET_LEN].data.cpu().numpy())]
        c3 = ALL_CHAR_SET[np.argmax(
            predict_label[0, 3 * ALL_CHAR_SET_LEN:4 * ALL_CHAR_SET_LEN].data.cpu().numpy())]
        predict_label = '%s%s%s%s' % (c0, c1, c2, c3)
        true_label = decode(labels.numpy()[0])
        total += labels.size(0)
        if(predict_label == true_label):
            correct += 1

    return 100 * correct / total

+ VGG16模型

  单通道

In [0]:
class CNN2(nn.Module):
    def __init__(self):
        super(CNN2, self).__init__()
        '''输入改为3，因为RGB3通道，原来是1'''
        self.layer1 = self.make_layers([64, 64, 'M'], 1, droup_out=False) 
        self.layer2 = self.make_layers([128, 128, 'M'], 64, droup_out=True)
        self.layer3 = self.make_layers(
            [256, 256, 256, 'M', 512, 512, 512, 'M'], 128, droup_out=False)
        self.layer4 = self.make_layers(
            [512, 512, 512, 'M'], 512, droup_out=True)
        self.fc = nn.Sequential(
            nn.Linear((IMAGE_WIDTH//32)*(IMAGE_HEIGHT//32)*512, 1024),
            nn.Dropout(0.5),  # drop 50% of the neuron
            nn.LeakyReLU())
        self.rfc = nn.Sequential(nn.Linear(1024, MAX_CAPTCHA*ALL_CHAR_SET_LEN))

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.view(out.size(0), -1) #-1, out.size(0)
        # print("-->{}".format(out.size()))
        out = self.fc(out)
        out = self.rfc(out)
        return out

    def make_layers(self, cfg, inc, droup_out=False):
        layers = []
        i = inc
        for v in cfg:
            if v == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                conv2d = nn.Conv2d(i, v, kernel_size=3, padding=1)
                if droup_out:
                    layers += [conv2d,
                               nn.BatchNorm2d(v), nn.Dropout(0.5), nn.LeakyReLU()]
                else:
                    layers += [conv2d, nn.BatchNorm2d(v), nn.LeakyReLU()]
                i = v
        return nn.Sequential(*layers)

    def model_name(self):
        return self.__class__.__name__

+ ResNet18模型

In [0]:
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=64):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.layer1 = self.make_layer(ResidualBlock, 64,  2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)
        # self.fc = nn.Linear(512, MAX_CAPTCHA*ALL_CHAR_SET_LEN)
        # self.fc = nn.Sequential(
        #     nn.Linear((IMAGE_WIDTH // 8) * (IMAGE_HEIGHT // 8) * 512, 512),
        #     nn.Dropout(0.5),  # drop 50% of the neuron
        #     nn.LeakyReLU())
        self.rfc = nn.Sequential(nn.Linear(512, MAX_CAPTCHA * ALL_CHAR_SET_LEN))
        self.drop = nn.Dropout(0.5)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)   #strides=[1,1]
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)                  # (batch_size, 512, 5, 15)
        out = nn.AdaptiveAvgPool2d(1)(out)      # (batch_size, 512, 1, 1)
        out = out.view(-1, 512)                 # (batch_size, 512)
        out = self.drop(out)
        out = self.rfc(out)                     # 248
        return out
    
    def model_name(self):
        return self.__class__.__name__

def ResNet18():
    return ResNet(ResidualBlock)

+ 训练模型

In [0]:
# 如果路径不存在，创建
if not os.path.isdir("./checkpoint"):
    os.mkdir("./checkpoint")

# 断点续训标识
RESUME = False
checkpoint_path = r'./checkpoint/ckpt_ResNet18_second_times-model.pth' #r'./checkpoint/ckpt_model.pth'

# Hyper Parameters
num_epochs = 100
# num_epochs = 1
# batch_size = 100
batch_size = 100
learning_rate = 3e-4
acc_max=0

# Train the Model
train_dataloader = get_train_data_loader(batch_size)

model = ResNet18() # CNN2()
model.train()

# 模型移入GPU
if torch.cuda.is_available():
    model = model.cuda()
print('Init ## {} ## model.'.format(model.model_name()))
# print(model)

# 开始训练的epoch
start_epoch = -1

# 如果RESUME==True，加载已保存的模型
if RESUME:
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    start_epoch = checkpoint['start_epoch']
    acc_max=checkpoint['acc']
    #loss=checkpoint['loss']
    print('Continue training from epoch {}...'.format(start_epoch))

criterion = nn.MultiLabelSoftMarginLoss().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)    

#print('start_epoch+1:',start_epoch+1,'载入模型的acc:',checkpoint['acc'],"载入模型的loss",checkpoint['loss'])
for epoch in range(start_epoch+1 ,num_epochs):
    for i, (images, labels) in enumerate(train_dataloader):
        # 数据移入GPU
        images = images.cuda()
        labels = labels.cuda()
        predict_labels = model(images)
        loss = criterion(predict_labels, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 10 == 0:
            print("epoch:", epoch, "step:", i+1, "loss:", loss.item())

        # if (i+1) % 100 == 0:
        #     torch.save(model.state_dict(), "model.pkl")   #current is model.pkl
        #     print("save model")
        numCor = test_data(model) #函数返回的是准确率
        pthname = "{0}_epoch{1}_loss{2}_acc_{3}.pth".format(
            model.model_name(), epoch, str(loss.item())[:8], numCor)

    if numCor>acc_max:
      checkpoint = {
        'model':model.state_dict(),
        'optimizer':optimizer.state_dict(),
        'start_epoch': epoch,
        'loss': loss.item(),
        'acc': numCor
      }
      acc_max=numCor
      torch.save(checkpoint, './checkpoint/ckpt_ResNet18_6.2-model.pth')
      print("save model:"+pthname)
      print('model loss:',checkpoint['loss'],'model acc:',checkpoint['acc'])
    else:
      print('no save, acc_max:',acc_max, '当前acc:',numCor,'loss:',loss.item())
    print("epoch:", epoch, "finished!", "loss:", loss.item())

    #if (epoch) % 5 == 0 and epoch != 0:

    # numCor = test_data(model) #函数返回的是准确率
    # pthname = "{0}_epoch{1}_loss{2}_acc_{3}.pth".format(
    #     model.model_name(), epoch, str(loss.item())[:8], numCor)

    # if numCor>acc_max:
      
    #   checkpoint = {
    #     'model':model.state_dict(),
    #     'optimizer':optimizer.state_dict(),
    #     'start_epoch': epoch,
    #     'loss': loss.item(),
    #     'acc': numCor
    #   }

    #   acc_max=numCor
    #   torch.save(checkpoint, './checkpoint/ckpt_ResNet18_second_times-model.pth')
    #   print("save model:"+pthname)
    #   print('model loss:',checkpoint['loss'],'model acc:',checkpoint['acc'])
    # else:
    #   print('no save, acc_max:',acc_max, '当前acc:',numCor,'loss:',loss.item())
    
#torch.save(model.state_dict(), "./model/{0}.pkl".format(model.model_name()))
# torch.save(model.state_dict(), "model.pkl")               #current is model.pkl
torch.save(checkpoint, './checkpoint/ckpt_ResNet18_last.pth')
print("save last model")

Init ## ResNet ## model.


In [0]:
!nvidia-smi
#os.mkdir("./checkpoint")

+ 测试模型

In [0]:
model = ResNet18()
model.eval()
model.load_state_dict(torch.load('checkpoint/ckpt_ResNet18_second_times-model.pth')['model'])
print("load model net.")

test_dataloader = get_test_data_loader()
print('Test Size:', len(test_dataloader.dataset))
correct = 0
total = 0
for i, (images, labels) in enumerate(test_dataloader):
    # vimage = Variable(image)
    predict_label = model(images)

    c0 = ALL_CHAR_SET[np.argmax(predict_label[0, 0:ALL_CHAR_SET_LEN].data.numpy())]
    c1 = ALL_CHAR_SET[np.argmax(predict_label[0, ALL_CHAR_SET_LEN:2 * ALL_CHAR_SET_LEN].data.numpy())]
    c2 = ALL_CHAR_SET[np.argmax(predict_label[0, 2 * ALL_CHAR_SET_LEN:3 * ALL_CHAR_SET_LEN].data.numpy())]
    c3 = ALL_CHAR_SET[np.argmax(predict_label[0, 3 * ALL_CHAR_SET_LEN:4 * ALL_CHAR_SET_LEN].data.numpy())]
    predict_label = '%s%s%s%s' % (c0, c1, c2, c3)
    true_label = decode(labels.numpy()[0])
    # print(predict_label, true_label)
    total += labels.size(0)
    if predict_label == true_label:
        correct += 1
    if total % 200 == 0:
        print('Test Accuracy of the model on the %d test images: %f %%' % (total, 100 * correct / total))
print('Test Accuracy of the model on the %d test images: %f %%' % (total, 100 * correct / total))

load model net.
Test Size: 2000
Test Accuracy of the model on the 200 test images: 98.500000 %
Test Accuracy of the model on the 400 test images: 96.750000 %
Test Accuracy of the model on the 600 test images: 95.500000 %
Test Accuracy of the model on the 800 test images: 95.125000 %
Test Accuracy of the model on the 1000 test images: 94.400000 %
Test Accuracy of the model on the 1200 test images: 94.416667 %
Test Accuracy of the model on the 1400 test images: 94.214286 %
Test Accuracy of the model on the 1600 test images: 94.250000 %
Test Accuracy of the model on the 1800 test images: 94.500000 %
Test Accuracy of the model on the 2000 test images: 94.600000 %
Test Accuracy of the model on the 2000 test images: 94.600000 %


+ 预测

In [0]:
import csv
csv_file = 'submission.csv'
# predict文件的路径
# os.mkdir
pathDir = os.listdir('dataset/test/')

f = open(csv_file, "w", newline='')
csv_writer = csv.writer(f)
csv_writer.writerow(["ID","label"])

# cnn = CNN()           # 采用模型1
model = ResNet18()            # 采用模型2
model.eval()
model.load_state_dict(torch.load('checkpoint/ckpt_ResNet18_model.pth')['model'])
print("load model net.")

# 按照文件名（除去格式部分）排序
pathDir.sort(key=lambda x: int(x[:-4]))
print('pathDir:',pathDir)
for allDir in pathDir:
    file = os.path.join('%s%s' % ('dataset/test/', allDir))
    print('file:',file)
    fopen = Image.open(file)

    image = fopen.resize((120, 40), Image.BICUBIC)
    image = transform(image)
    image = torch.unsqueeze(image, dim=0)

    predict_label = model(image)

    c0 = ALL_CHAR_SET[np.argmax(predict_label[0, 0:ALL_CHAR_SET_LEN].data.numpy())]
    c1 = ALL_CHAR_SET[np.argmax(
        predict_label[0, ALL_CHAR_SET_LEN:2 * ALL_CHAR_SET_LEN].data.numpy())]
    c2 = ALL_CHAR_SET[np.argmax(
        predict_label[0, 2 * ALL_CHAR_SET_LEN:3 * ALL_CHAR_SET_LEN].data.numpy())]
    c3 = ALL_CHAR_SET[np.argmax(
        predict_label[0, 3 * ALL_CHAR_SET_LEN:4 * ALL_CHAR_SET_LEN].data.numpy())]
    label = '%s%s%s%s' % (c0, c1, c2, c3)

    ID = str(allDir)
    csv_writer.writerow([ID, label])
    print(ID, label)

f.close()